# Testing Notebook

### Imports and Data Input

In [23]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate
import time
# import data_files

### Dashboard of Chosen Run

# Testing Class

In [24]:
import configparser

class testing():

    def __init__(self):
        self.data = {
            'sog': 0,               # Knot, COG SOG Rapid Update
            'rpm': 0,               # RPM, Engine Parameters Rapid Update
            'date': "12:34 PM",     # GNSS Position Data
            'time': "12:34 PM",     # GNSS Position Data
            'cog': 0,               # Deg, COG SOG Rapid Update
            'heading': "N",         # calculated
            'numFaults': 0,         # TODO
            'faults': "",           # TODO
            'power': 0.0,           # kW, calculated
            'tripDistance': 0.0,    # miles
            'soc': 0,               # %, DC Detailed Status
            'energyUsed': 0.0,      # kWh
            'battCycles': 0,        # stored locally
            'motorTemp': 0,         # degC, Engine Parameters Dynamic
            'totalDistance': 0,     # miles
            'minsRemaining': 0,     # mins, DC Detailed Status
            'tripDuration': 0,      # mins
            'motorTorquePct': 0,    # %, Engine Parameters Dynamic
            'packVoltage': 0,       # V, DC Voltage Current
            'packCurrent': 0.0,     # A, DC Voltage Current
            'packTemp': 0,          # degC, Battery Status
            'soh': 0,               # %, DC Detailed Status
            'totalMotorHours': 0.0, # hours motor rpm > 0
            'motorVoltage': 0,      # V, Engine Parameters Dynamic (alternator potential /10)
            'gear': "Unknown",      # Neutral, Forward, Reverse
            'motorEnabled': 0,      # 0: disabled, 1: enabled
            'nmea2000_timeout': 0,  # 0: NMEA2000 data received (EV), 1: timeout
            'state_sleep': 0,
            'state_acc': 0,
            'state_ign': 0,
            'state_run': 0,
            'state_charge': 0
            }
        
    #     self.config = configparser.ConfigParser()
    #     self.load_nvm()
    #     self.last_distance_algo_update = None
    #     self.distance_algo_interval_ms = 5000


    # def load_nvm(self):
    #     inifile = self.config.read(NAME.lower()+'.nvm')

    #     if len(inifile) == 0:
    #         self.config[NAME] = {'batt_cycles': '0', 'total_distance': '0.0', 'total_motor_hours': '0.0', 'total_motor_hours3': '0.0'}
    #         self.persist_nvm()
    #         inifile = self.config.read(NAME.lower()+'.nvm')

    #     self.data['battCycles'] = int(self.config[NAME]['batt_cycles'])
    #     self.data['totalDistance'] = float(self.config[NAME]['total_distance'])
    #     self.data['totalMotorHours'] = float(self.config[NAME]['total_motor_hours'])
    #     self.data['totalMotorHours3'] = float(self.config[NAME]['total_motor_hours3'])

    # def persist_nvm(self):
    #     with open(NAME.lower()+'.nvm', 'w') as configfile:
    #         self.config.write(configfile)
    #         configfile.close()

    # def update_nvm(self):
    #     self.config[NAME]['total_distance'] = str(self.data['totalDistance'])
    #     self.config[NAME]['total_motor_hours'] = str(self.data['totalMotorHours'])
    #     self.config[NAME]['total_motor_hours3'] = str(self.data['totalMotorHours3'])
    #     self.config[NAME]['batt_cycles'] = str(self.data['battCycles'])
        
    #     self.persist_nvm()

    # def run_distance_algo(self):
    #     # Run the distance remain algo here
    #     print(self.data)



    def add_variables(self, df):
        """Add calculated variables to testing DataFrame which show up in NMEA Server but aren't in CSV files"""
        
        df['Time'] = pd.to_datetime(df['Time'], format="%H:%M:%S")
        df['Distance nm'] = df['Distance km']*0.539957
        df['tripDistance'] = ''
        df['tripDuration'] = ''
        df['energyUsed'] = ''
        df['energyAvailable'] = ''

        for i in range(len(df)):
            df['tripDistance'].iloc[i] = (df['Distance km'].iloc[i] - df['Distance km'].iloc[0])*.539957    # nautical miles
            df['tripDuration'].iloc[i] = (df['Time'].iloc[i] - df['Time'].iloc[0]).seconds/60               # min
            df['energyUsed'].iloc[i] = (df['SOC 1 %'].iloc[0] - df['SOC 1 %'].iloc[i])*58/100               # kWh
            df['energyAvailable'].iloc[i] = df['SOC 1 %'].iloc[i]*58/100                                    # kWh
        return df


    def parse_csv(self, df):
        """Interpret DataFrame values as data class values to replicate NMEA Server"""

        self.data['sog'] = df['Speed m/s']*1.94384          # knots
        self.data['time'] = df['Time']
        self.data['totalDistance'] = df['Distance km']*0.539957
        self.data['soc'] = df['SOC 1 %']
        self.data['packVoltage'] = df['Pack Voltage 1 V']*10
        self.data['packCurrent'] = df['Pack Current 1 A']

        '''Missing Variables'''
        self.data['tripDistance'] = df['tripDistance']      # manually added in test loop
        self.data['energyUsed'] = df['energyUsed']          # manually added in test loop
        self.data['energyAvailable'] = df['energyAvailable']          # manually added in test loop
        self.data['tripDuration'] = df['tripDuration']      # manually added in test loop

        '''Calculated Columns'''
        self.data['power'] = self.data['packVoltage'] * self.data['packCurrent'] / 1000.0   # kW    

        return self.data

### Define Algorithms

In [25]:
class range_est():

    def __init__(self, battery_capacity):
        self.max_battery = battery_capacity                         # kWh    


    def overall_dist_avg(self, data, cached_avg):
        """This function evaluates range remaining on the battery given historical distance consumption data (kWh/nm) from all past trips."""
        range_remaining = data['energyAvailable']/cached_avg        # nautical miles

        return range_remaining


    def overall_time_avg(self, data, cached_avg):
        """This function evaluates range remaining on the battery given historical time consumption data (kWh/min) from all past trips."""
        time_remaining = data['energyAvailable']/cached_avg         # min 
        range_remaining = (time_remaining/60)*data['sog']           # nm

        return time_remaining, range_remaining


    def rolling_avg(self, cached_avg, N_minutes):
        """This function updates the range based on the trip duration. 
        Range will be calculated based on cached_avg (cached average energy consumption) for the first N minutes of the trip.
        After N minutes, range will be calculated with the average consumption for the given trip."""

        # Need a way to update rolling average every N minutes
        roll_consumption = self.data['energyUsed']/self.data['tripDistance']        # kWh/nm

        if self.data['tripDuration'] < N_minutes or roll_consumption==0:
            range_remaining = self.data['energyAvailable']/cached_avg
        else:
            # Insert a weighting function here (logarithmic?)
            range_remaining = self.data['energyAvailable']/roll_consumption         # nm

        return range_remaining, roll_consumption
    

    def update_avg(self, data, cached_avg, nRuns):
        """This function should be called whenever a trip is completed, updating the cached average.
        It stores the average consumption rate as well as the number of runs it is averaged over."""
        trip_avg = data['energyUsed']/data['tripDistance']                          # kWh/nm
        new_avg = (nRuns*cached_avg + trip_avg)/(nRuns+1)
        cached_avg = new_avg
        nRuns += 1

        return cached_avg, nRuns


# Testing

In [26]:
"""Import Data from file manager"""
# data = data_files.runs_dict
# run = 31
# df = data['Run %i' % run]

"""Alternately, direct import from csv file"""
file_path, rows_to_read = 'data/L230414.CSV', range(300, 3300)
df = pd.read_csv(file_path, skiprows=range(1,rows_to_read[0]), nrows=len(rows_to_read))

In [27]:
# '''Summary Plots'''
# plt.figure(figsize=(12,5))

# plt.subplot(2,2,1)
# plt.plot(df.index, df['SOC 1 %'])
# plt.grid()
# plt.title('Charge (%) over Time')

# plt.subplot(2,2,2)
# plt.plot(df.index, df['Speed m/s']) #, df['Power 1 kW'])
# plt.grid()
# plt.title('Power (kW) and Speed (kts) over Time')

# plt.subplot(2,2,3)
# plt.plot(df.index, df['Pack Voltage 1 V'], df['Pack Current 1 A'])
# plt.grid()
# plt.title('Current and Voltage over Time')

# # plt.subplot(2,2,4)
# # plt.plot(df.index)
# # plt.grid()
# # plt.title('Empty')

# plt.subplots_adjust(hspace = 0.5)
# plt.subplots_adjust(wspace = 0.3)

# df.head()

In [28]:
'''Add missing variables to DataFrame'''
range_estimator = range_est(58)
df = testing().add_variables(df)
cached_avg = 4.5

dist_list = []
"""Test Loop"""
for i in range(len(df)):
    dataStream = testing().parse_csv(df.iloc[i])
    range_remaining = range_estimator.overall_dist_avg(dataStream, cached_avg)
    print('Battery Remaining = %.1f percent | Range Remaining = %.1f nm' % (dataStream['soc'], range_remaining), end=' \r')
    time.sleep(.001)

    dist_list.append(range_remaining)

/tmp/ipykernel_346/4033712971.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tripDistance'].iloc[i] = (df['Distance km'].iloc[i] - df['Distance km'].iloc[0])*.539957    # nautical miles
/tmp/ipykernel_346/4033712971.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tripDuration'].iloc[i] = (df['Time'].iloc[i] - df['Time'].iloc[0]).seconds/60               # min
/tmp/ipykernel_346/4033712971.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

# Accuracy Comparison

In [38]:
'''Look at how far the vessel traveled 30s later'''

df['Dist Prediction (nm)'] = dist_list

N = range(500, len(df), 500)
for n in N:
    pred = df['Dist Prediction (nm)'].iloc[n] - df['Dist Prediction (nm)'].iloc[-1]     # predicted distance remaining at n minus prediction at the end
    dist = df['Distance nm'].iloc[-1] - df['Distance nm'].iloc[n]                       # actual distance traveled since n
    print(n, 'Predicted:', round(pred, 2), '| Actual:', round(dist, 2))


500 Predicted: 5.03 | Actual: 4.76
1000 Predicted: 3.74 | Actual: 3.86
1500 Predicted: 2.58 | Actual: 3.03
2000 Predicted: 1.68 | Actual: 2.07
2500 Predicted: 0.26 | Actual: 0.76
